# Predicting train delays and detecting unusual service patterns



In [ ]:
import pandas as pd
import numpy as np
import json
from requests.auth import HTTPBasicAuth
from datetime import datetime, date
import os

In [2]:
pd.set_option('display.max_colwidth', None)

## Collecting service data

In [ ]:
start_date = date(2024, 11, 24)
end_date = date(2025, 11, 24)

